<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/%5Bcontextual_bugs%5D_T5_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 2.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.1 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation:

In [2]:
from google.colab import auth
from google.auth import default
from numpy import random
import pandas as pd
import gspread
import gc
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
#defining my worksheet
worksheet = gc.open('contextual_bugs').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
cols = ['Source', 'Target']
df = pd.DataFrame(rows, columns=cols)

In [3]:
df['Source']

0     1 //SPDX-License-Identifier: GPL-3.0\n2 pragma...
1     1 // SPDX-License-Identifier: MIT\n2 pragma so...
2     1  // SPDX-License-Identifier: MIT\n2  pragma ...
3     1 // SPDX-License-Identifier: MIT\n2 pragma so...
4     1 // SPDX-License-Identifier: MIT\n2 pragma so...
5     1 // SPDX-License-Identifier: MIT\n2 \n3 pragm...
6                     4+ revert(unique ID violation!); 
7     1 pragma solidity ^0.4.11;\n2 \n3 \n4 import "...
8     1 pragma solidity ^0.4.11;\n2 \n3 \n4 contract...
9     1 pragma solidity ^0.4.11;\n2 \n3 \n4 import "...
10    1 pragma solidity ^0.4.11;\n2 \n3 contract Vot...
11    1 // SPDX-License-Identifier: MIT\n2 \n3 pragm...
12    1 // SPDX-License-Identifier: MIT\n2 pragma so...
13    1 // SPDX-License-Identifier: MIT\n2 pragma so...
14    1  // SPDX-License-Identifier: MIT\n2  pragma ...
15    1 // SPDX-License-Identifier: MIT\n2  pragma s...
16    1 // SPDX-License-Identifier: GPL-3.0-or-later...
17    1 // SPDX-License-Identifier: GPL-3.0-or-l

In [4]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

def plot_loss(index_list, loss_list):
  results = {
      "epochs": index_list,
      "cross entropy loss": loss_list,
  }
  df = pd.DataFrame(results)
  fig = px.line(df, x ="epochs", y="cross entropy loss",  title="Evaluation")
  fig.show()

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Cross Entropy Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [5]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())
    
    source = self.tokenizer.batch_encode_plus([source_text], \
                                              max_length= self.source_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                            #  padding='longest',\
                                              return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], \
                                              max_length= self.summ_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                           #   padding='longest', \
                                              return_tensors='pt')   
    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [7]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  loss_list = []
  for _, data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)
    outputs = model(input_ids = ids, attention_mask = mask, \
                    decoder_input_ids=y_ids, labels=lm_labels)  
    total_loss = float(outputs[0].item())
    loss_list.append(total_loss)
    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(total_loss))
      console.print(training_logger)

    optimizer.zero_grad()
    outputs[0].backward()
    optimizer.step()
    return loss_list



In [8]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=700, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=5.0, 
              early_stopping=True
              )
          print("generated token length: \n", len(generated_ids[0]))
          preds = [tokenizer.decode(g, skip_special_tokens=True, \
                                    clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, \
                                     clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')
          predictions.extend(preds)
          actuals.extend(target)
  print('predictions: \n', predictions)
  print('actuals: \n', actuals)
  return predictions, actuals

In [9]:
import gc
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./output/"):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  gc.collect()
  torch.cuda.empty_cache()
  os.environ['CUDA_VISIBLE_DEVICES']='0, 1, 2, 3'
  os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params = model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  loss_result = []
  epoch_list = []
  for epoch in range(model_params["TRAIN_EPOCHS"]):
      loss = train(epoch, tokenizer, model, device, training_loader, optimizer)
      loss_result.extend(loss)
      epoch_list.append(epoch)

  plot_loss(epoch_list, loss_result)
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
model_params={
    "MODEL":"t5-small",             # model_type: t5-large
    "TRAIN_BATCH_SIZE": 8,          # training batch size: 
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS": 200,              # number of training epochs:20 seems optimal based on experiments
    "VAL_EPOCHS":3,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate: can be 1e-4
    "MAX_SOURCE_TEXT_LENGTH": 1000,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 1000,   # max length of target text
    "SEED": random.randint(1000)    # randomized seeds to shuffle test set

}

In [12]:
#To solve CUDA out of memory error
import gc
gc.collect()
torch.cuda.empty_cache()
os.environ['CUDA_VISIBLE_DEVICES']='0, 1, 2, 3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [13]:
T5Trainer(dataframe=df, source_text="Source", target_text="Target", model_params=model_params, output_dir="outputs")

[08:59:23] [Model]: Loading t5-small...                                           <ipython-input-9-c6bf7c73f73a>:15
                                                                                                                   

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[08:59:34] [Data]: Reading data...                                                <ipython-input-9-c6bf7c73f73a>:30
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                            source_text                              |                target_text                |
|---------------------------------------------------------------------+-------------------------------------------|
|                1 //SPDX-License-Identifier: GPL-3.0                 |  82+ revert('price manipulation attack!); |
|                      2 pragma solidity 0.8.4;                       | 150+  revert('price manipulation attack!);|
|                                 3                                   | 152+  revert('price manipulation attack!);|
|                4 import "../libraries/MathLib.sol";                 |                                           |
|     5 import "@openzeppelin/contracts/token/ERC20/ERC20.sol";       |                                           |
|     6 import "@openzeppelin/contracts/token/ERC20/IERC20.sol";      |                                           |
|7 import "@openzeppelin/contracts/token/ERC20/utils/SafeERC20.sol";  |                                           |
|  8 import "@openzeppelin/contracts/security/ReentrancyGuard.sol";   |                                           |
|           9 import "../interfaces/IExchangeFactory.sol";            |                                           |
|                                 10                                  |                                           |
|                                 11                                  |                                           |
|          12 contract Exchange is ERC20, ReentrancyGuard {           |                                           |
|                 13     using MathLib for uint256;                   |                                           |
|                 14     using SafeERC20 for IERC20;                  |                                           |
|                                 15                                  |                                           |
|16     address public immutable baseToken; // address of ERC20 base  |                                           |
|                  token (elastic or fixed supply)                    |                                           |
|  17     address public immutable quoteToken; // address of ERC20    |                                           |
|        quote token (WETH or a stable coin w/ fixed supply)          |                                           |
|      18     address public immutable exchangeFactoryAddress;        |                                           |
|                                 19                                  |                                           |
|  20     uint256 public constant TOTAL_LIQUIDITY_FEE = 30; // fee    |                                           |
|       provided to liquidity providers + DAO in basis points         |                                           |
|                                 21                                  |                                           |
|     22     MathLib.InternalBalances public internalBalances =       |                                           |
|           23         MathLib.InternalBalances(0, 0, 0);             |                                           |
|                                 24                                  |                                           |
|                     25     event AddLiquidity(                      |                                           |
|           26         address indexed liquidityProvider,             |                                           |
|               27         uint256 baseTokenQtyAdded,                 |                                           |
|               28         uint256 quoteTokenQtyAdded   

FULL Dataset: (19, 2)

...

predictions: 
 ['+ revert(_expirationTimestamp).45+ revert(../libraries/MathLib.sol");', '+ revert(atomicity flaw!);', '57+ revert(business logic flaw!); 47+ revert(business logic flaw!);', '19+ revert(business logic flaw!);']
actuals: 
 ["82+ revert('price manipulation attack!); 150+ revert('price manipulation attack!); 152+ revert('price manipulation attack!);", "75+ revert('price manipulation attack!);", '29+ revert(business logic flaw!); 69+ revert(business logic flaw!);', '19+ revert(business logic flaw!);']


[09:01:17] [Validation Completed.]                                               <ipython-input-9-c6bf7c73f73a>:105
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt

In [14]:

test_params = {
      'batch_size': 8,
      'shuffle': False,
      'num_workers': 0
      }

class TestDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, source_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.source_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    print('source_text: ', source_text)
    source = self.tokenizer.batch_encode_plus([source_text], \
                                              max_length=self.source_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                              padding="max_length",\
                                              return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
    }

In [139]:
test_contract = 'sushiTrident.sol'

In [140]:

import gc
#let's get the trained model and never-seen test contracts 
def initialize():
  gc = gspread.authorize(creds)
  trained_model = T5ForConditionalGeneration.from_pretrained("outputs/model_files")
  tokenizer = T5Tokenizer.from_pretrained("t5-small")
  col = ['Test']
  f = open(f'/content/drive/MyDrive/experiments/baseline benchmark/contextual_bugs_test/{test_contract}', "r")
  file = f.read()
  test_df = pd.DataFrame([file], columns=col)
  #Convert to a DataFrame 
  return test_df, trained_model, tokenizer

#test how the model performs on never-seen test contracts
def generate(df, model, tokenizer):
   model.eval()
   test_val = TestDataSetClass(df, tokenizer, source_len=512, source_text="Test")
   test_loader = DataLoader(test_val, **test_params)
   predictions = []
   with torch.no_grad():
      for _, data in enumerate(test_loader, 0):
          ids = data['source_ids']
          mask = data['source_mask'] #.to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, \
                                    clean_up_tokenization_spaces=True) for g in generated_ids]
          print("preds: ", preds)
          if _%10==0:
              console.print(f'Completed {_}')
          predictions.extend(preds)
         # print('predictions: \n', predictions)
   return predictions



In [141]:
df2, trained_model, tokenizer = initialize()

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning:

This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.



In [142]:
out = generate(df2, trained_model, tokenizer)
print(out)

source_text:  1 // SPDX-License-Identifier: GPL-3.0-or-later 2 3 pragma solidity >=0.8.0; 4 5 import "../../interfaces/IBentoBoxMinimal.sol"; 6 import "../../interfaces/IMasterDeployer.sol"; 7 import "../../interfaces/IPool.sol"; 8 import "../../interfaces/IPositionManager.sol"; 9 import "../../interfaces/ITridentCallee.sol"; 10 import "../../interfaces/ITridentRouter.sol"; 11 import "../../libraries/concentratedPool/FullMath.sol"; 12 import "../../libraries/concentratedPool/TickMath.sol"; 13 import "../../libraries/concentratedPool/UnsafeMath.sol"; 14 import "../../libraries/concentratedPool/DyDxMath.sol"; 15 import "../../libraries/concentratedPool/SwapLib.sol"; 16 import "../../libraries/concentratedPool/Ticks.sol"; 17 import "hardhat/console.sol"; 18 19 20 contract ConcentratedLiquidityPool is IPool { 21 using Ticks for mapping(int24 => Ticks.Tick); 22 23 event Mint(address indexed sender, uint256 amount0, uint256 amount1, address indexed recipient); 24 event Burn(address indexed s

Completed 0

['(inconsistent state update!);']


In [143]:
def get_line_and_invariant(prediction):  
  line_df = []
  for item in prediction:   
    for line in item.split(';'):
      line_df.append(line)
  invariants = []
  for i in range(len(line_df)-1): 
      letter_index = 0
      letter = line_df[i][0]
      line_number = ''
      invariant = ''
      while letter != '+':  
        line_number += letter 
        letter_index += 1  
        if letter_index > (len(line_df[i]) - 1):
          break        
        letter = line_df[i][letter_index]        
        if letter_index >= 5:
          line_number = ''        
          break 
      if letter_index >= 5:
        line_number = '' 
        invariant = line_df[i][0:len(line_df[i])] 
      else: 
        invariant = line_df[i][letter_index+1:len(line_df[i])]
      invariants.append([line_number.replace(" ", ""), invariant])
  return invariants

def insert_and_annotate(test_contract, invariants): 
  new_template = open(f'/content/drive/MyDrive/experiments/evaluation/T5_annotated_contracts/{test_contract}(annotated)', "w")
  for line in df2['Test'][0].split('\n'): 
      if line != '':   
        for item in invariants: 
          if item[0] == line[:4].replace(" ", "") and item[0] != '':            
            comment = '//[SCInvarinfer] inserting inferred invarint below: \n'
            line = '\n'+ comment + line + '\n' + '\t' + item[1] + ';'
            new_template.write(line + '\n')    
      new_template.write(line + '\n')
      print(line)
  new_template.close()

In [144]:
invariants = get_line_and_invariant(out)

def save_invariants(test_contract, invariants):
  new_template = open(f'/content/drive/MyDrive/experiments/evaluation/T5_inferred_invariants/{test_contract}(invariants)', "w")
  for line in invariants:       
      new_template.write(str(line) + '\n')
      print(line)
  new_template.close()


In [145]:
save_invariants(test_contract, invariants)

['', '(inconsistent state update!)']


In [146]:
insert_and_annotate(test_contract, invariants)

1 // SPDX-License-Identifier: GPL-3.0-or-later
2 
3 pragma solidity >=0.8.0;
4 
5 import "../../interfaces/IBentoBoxMinimal.sol";
6 import "../../interfaces/IMasterDeployer.sol";
7 import "../../interfaces/IPool.sol";
8 import "../../interfaces/IPositionManager.sol";
9 import "../../interfaces/ITridentCallee.sol";
10 import "../../interfaces/ITridentRouter.sol";
11 import "../../libraries/concentratedPool/FullMath.sol";
12 import "../../libraries/concentratedPool/TickMath.sol";
13 import "../../libraries/concentratedPool/UnsafeMath.sol";
14 import "../../libraries/concentratedPool/DyDxMath.sol";
15 import "../../libraries/concentratedPool/SwapLib.sol";
16 import "../../libraries/concentratedPool/Ticks.sol";
17 import "hardhat/console.sol";
18 
19 
20 contract ConcentratedLiquidityPool is IPool {
21     using Ticks for mapping(int24 => Ticks.Tick);
22 
23     event Mint(address indexed sender, uint256 amount0, uint256 amount1, address indexed recipient);
24     event Burn(address indexe

In [147]:
#for downloading purpose 
#!zip -r /content/T5-model.zip /content/outputs/model_files

In [148]:
#from google.colab import files
#files.download("/content/T5-model.zip")